<a href="https://colab.research.google.com/github/pavithranrao/dlwpt-code/blob/master/p1ch2/4_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
torch.manual_seed(4242)

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data/p1ch2/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

Extracting ../data/p1ch2/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw


Extracting ../data/p1ch2/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw



Extracting ../data/p1ch2/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw


Extracting ../data/p1ch2/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/p1ch2/mnist/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# nn.MaxPool2d?

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Net()

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    print('Current loss', float(loss))

Current loss 0.4458557367324829
Current loss 0.24386711418628693
Current loss 0.39750704169273376
Current loss 0.396612286567688
Current loss 0.29553699493408203
Current loss 0.1466696858406067
Current loss 0.08304823935031891
Current loss 0.1304263025522232
Current loss 0.3869445323944092
Current loss 0.19635553658008575


In [ ]:
torch.save(model.state_dict(), '../data/p1ch2/mnist/mnist.pth')

In [ ]:
pretrained_model = Net()
pretrained_model.load_state_dict(torch.load('../data/p1ch2/mnist/mnist.pth'))

In [11]:
torch.einsum?